Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf


#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'AAPL'

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo('AAPL', '2020-10-31', '2021-10-31')
df = df.iloc[:, :4]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2020-10-30,111.059998,111.989998,107.720001,108.860001
2020-11-02,109.110001,110.680000,107.320000,108.769997
2020-11-03,109.660004,111.489998,108.730003,110.440002
2020-11-04,114.139999,115.589996,112.349998,114.949997
2020-11-05,117.949997,119.620003,116.870003,119.029999
...,...,...,...,...
2021-10-25,148.679993,149.369995,147.619995,148.639999
2021-10-26,149.330002,150.839996,149.009995,149.320007
2021-10-27,149.360001,149.729996,148.490005,148.850006


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window = 15).mean()
df['Open_1_Change'] = df['Open'] - df['Open'].shift(1)
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2020-11-19,117.589996,119.059998,116.809998,118.639999,116.490000,-1.020004
2020-11-20,118.639999,118.769997,117.290001,117.339996,117.055333,1.050003
2020-11-23,117.180000,117.620003,113.750000,113.849998,117.394000,-1.459999
2020-11-24,113.910004,115.849998,112.589996,115.169998,117.709333,-3.269997
2020-11-25,115.550003,116.750000,115.169998,116.029999,117.781333,1.639999
...,...,...,...,...,...,...
2021-10-25,148.679993,149.369995,147.619995,148.639999,144.967331,-1.010010
2021-10-26,149.330002,150.839996,149.009995,149.320007,145.514665,0.650009
2021-10-27,149.360001,149.729996,148.490005,148.850006,145.971332,0.029999


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(df['Close'].shift(-1) - df['Close'] >= 0, 1, -1)
y

array([-1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1,  1,
       -1,  1, -1,  1,  1, -1,  1,  1, -1, -1, -1, -1,  1, -1,  1,  1, -1,
       -1,  1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1,  1, -1,  1,
       -1,  1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1,  1,  1, -1,
       -1, -1,  1, -1,  1, -1,  1, -1,  1,  1, -1, -1, -1,  1, -1, -1,  1,
        1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1,  1, -1,  1, -1,
        1, -1,  1,  1, -1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1, -1,  1,
        1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1,
        1, -1,  1,  1, -1,  1,  1, -1,  1,  1, -1, -1, -1,  1,  1,  1,  1,
        1,  1,  1, -1,  1, -1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1, -1,
       -1,  1,  1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  1,  1, -1, -1,  1,
        1,  1, -1, -1, -1,  1,  1, -1,  1,  1,  1,  1, -1, -1, -1,  1, -1,
        1, -1, -1, -1,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1,
       -1, -1, -1, -1,  1

In [11]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train, X_test, y_train, y_test = X[:index], X[index:], y[:index], y[index:]

In [12]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

In [13]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[-0.07950713978945491]
1,High,[0.2903097216346121]
2,Low,[0.012319271734081266]
3,Close,[-0.22844407013331383]
4,Close_15_Rolling,[-0.051283828265663085]
5,Open_1_Change,[-0.0466294222961634]


In [14]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)
print(probabilities)
print(predictions)

[[0.75105675 0.24894325]
 [0.68942675 0.31057325]
 [0.77858262 0.22141738]
 [0.74467741 0.25532259]
 [0.70258455 0.29741545]
 [0.63557614 0.36442386]
 [0.73063145 0.26936855]
 [0.74777916 0.25222084]
 [0.74702162 0.25297838]
 [0.7372514  0.2627486 ]
 [0.76841437 0.23158563]
 [0.75060328 0.24939672]
 [0.73951416 0.26048584]
 [0.76410249 0.23589751]
 [0.69173682 0.30826318]
 [0.7490231  0.2509769 ]
 [0.75920814 0.24079186]
 [0.80204722 0.19795278]
 [0.76819356 0.23180644]
 [0.74194017 0.25805983]
 [0.59090938 0.40909062]
 [0.66315102 0.33684898]
 [0.79881209 0.20118791]
 [0.77443236 0.22556764]
 [0.75285053 0.24714947]
 [0.73037062 0.26962938]
 [0.72348645 0.27651355]
 [0.78355671 0.21644329]
 [0.7644823  0.2355177 ]
 [0.8118542  0.1881458 ]
 [0.69954885 0.30045115]
 [0.78915586 0.21084414]
 [0.80866685 0.19133315]
 [0.79973264 0.20026736]
 [0.78693268 0.21306732]
 [0.74725068 0.25274932]
 [0.5453599  0.4546401 ]
 [0.72487243 0.27512757]
 [0.71694812 0.28305188]
 [0.80255978 0.19744022]


In [16]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

print(metrics.confusion_matrix(y_test, predictions))

[[33  0]
 [39  0]]


In [17]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

print(logistic.score(X_test, y_test))

0.4583333333333333


In [18]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=5)
print(cross_val)
print(cross_val.mean())

[0.52083333 0.41666667 0.52083333 0.46808511 0.57446809]
0.500177304964539


C:\Users\Khloud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Khloud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite